![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
## Object detection and object counting in polygon zone: streaming video processing

This notebook is an example how to use DeGirum PySDK to do object detection and object 
counting in polygon zone while processing streaming video

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option.

When running this notebook locally, you need to specify your cloud API access token in the [env.ini](../../env.ini) file, located in the same directory as this notebook.

When running this notebook in Google Colab, the cloud API access token should be stored in a user secret named `DEGIRUM_CLOUD_TOKEN`.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools

#### Specify video file name, model name, and other options here

In [ ]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# model_name: name of the model for running AI inference
# video_source: video source for inference
#     camera index for local web camera
#     URL of RTSP stream
#     URL of YouTube Video
#     path to video file (mp4 etc)
# polygon_zones: zones in which objects need to be counted
# class_list: list of classes to be counted
# per_class_display: Boolean to specify if per class counts are to be displayed
hw_location = "@cloud"
model_zoo_url = "https://cs.degirum.com/degirum/public"
model_name = "yolo_v5s_coco--512x512_quant_n2x_orca1_1"
video_source = "https://github.com/DeGirum/PySDKExamples/raw/main/images/Traffic.mp4"
polygon_zones = [
    [[265, 260], [730, 260], [870, 450], [120, 450]],
    [[400, 100], [610, 100], [690, 200], [320, 200]],
]
class_list = ["car", "motorbike", "truck"]
per_class_display = True

#### The rest of the cells below should run without any modifications

In [ ]:
import degirum as dg, degirum_tools

# connect to AI inference engine
zoo = dg.connect(hw_location, model_zoo_url, degirum_tools.get_token())

# load model
model = zoo.load_model(model_name, overlay_line_width=1)

with degirum_tools.Display("AI Camera") as display:
    # create zone counter
    zone_counter = degirum_tools.ZoneCounter(
        polygon_zones,
        class_list=class_list,
        per_class_display=per_class_display,
        triggering_position=degirum_tools.AnchorPoint.CENTER,
        window_name=display.window_name,  # attach display window for interactive zone adjustment
    )

    # AI prediction loop
    # Press 'x' or 'q' to stop
    # Drag zone by left mouse button to move zone
    # Drag zone corners by right mouse button to adjust zone shape
    for inference_result in degirum_tools.predict_stream(
        model, video_source, analyzers=zone_counter
    ):
        display.show(inference_result)